<a href="https://colab.research.google.com/github/mcavol/AI-vs-Human-Text-Detector/blob/main/AI_vs_Human_Detector_with_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q --upgrade "transformers>=4.40" accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch, textwrap

MODEL_NAME = "openai-community/roberta-base-openai-detector"  # swap anytime

device_id = 0 if torch.cuda.is_available() else -1
clf = pipeline(
    "text-classification",
    model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME),
    tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME),
    return_all_scores=True,
    device=device_id,
)

LABEL = {"Real": "HUMAN", "Fake": "AI"}

def classify(text: str) -> tuple[str, float]:
    scores = clf(text, truncation=True, max_length=512)[0]
    best = max(scores, key=lambda s: s["score"])
    return LABEL[best["label"]], best["score"]

print("Paste text (empty line = quit):\n")
while True:
    try:
        snippet = input("> ").strip()
        if not snippet:
            break
        verdict, conf = classify(snippet)
        print(f"→ {verdict}  (confidence {conf:.1%})\n")
    except (EOFError, KeyboardInterrupt):
        break


Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Paste text (empty line = quit):

> I am a physicist transitioning into the fields of AI/ML and Data Science. I earned my Master's degree at École Polytechnique, France, one of the top 50 universities in the world. Then I worked on a PhD project in quantum optics at the Technical University of Munich, Germany, which ranks among the top 30 universities globally. During that time, I gained extensive experience in experimental work, as well as data analysis of measured experimental data.
→ HUMAN  (confidence 81.1%)

> While I may not be the ideal candidate due to my recent entry into the field, I  am eager to work hard and continuously learn to deepen my knowledge and  gain more experience
→ HUMAN  (confidence 93.9%)

> However, the joke creates a humorous image by taking the phrase literally. It suggests that Donald Trump, a figure famously associated with real estate and construction, has misinterpreted the saying. Instead of understanding that the drinks are free, the joke paints a pict